In [24]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 337 kB in 4s (90.4 kB/s)
Reading package lists... Done


In [25]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [26]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
import pandas as pd
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [27]:
  # 2. Create a temporary view of the DataFrame.
  home_sale_df = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv")
  home_sale_df.head()


,id,date,date_built,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view
0,f8a53099-ba1c-47d6-9c31-7398aa8f6089,2022-04-08,2016,936923,4,3,3167,11733,2,1,76
1,7530a2d8-1ae3-4517-9f4a-befe060c4353,2021-06-13,2013,379628,2,2,2235,14384,1,0,23
2,43de979c-0bf0-4c9f-85ef-96dc27b258d5,2019-04-12,2014,417866,2,2,2127,10575,2,0,0
3,b672c137-b88c-48bf-9f18-d0a4ac62fb8b,2019-10-16,2016,239895,2,2,1631,11149,2,0,0
4,e0726d4d-d595-4074-8283-4139a54d0d63,2022-01-08,2017,424418,3,2,2249,13878,2,0,4


In [28]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
import pandas as pd

import pandas as pd

# Create a Pandas DataFrame from the CSV file
home_sale_df = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv")

# Assuming 'date' is the correct column name representing the year
average_price_by_year = home_sale_df[home_sale_df['bedrooms'] == 4].groupby('date')['price'].mean().round(2).reset_index()

print(average_price_by_year)




            date      price
0     2019-01-01  365088.00
1     2019-01-02  271644.29
2     2019-01-03  381218.86
3     2019-01-04  359586.14
4     2019-01-05  323683.57
...          ...        ...
1256  2022-06-10  329435.69
1257  2022-06-11  301498.07
1258  2022-06-12  211683.20
1259  2022-06-13  329998.88
1260  2022-06-14  222403.00

[1261 rows x 2 columns]


In [29]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
import pandas as pd

# Create a Pandas DataFrame from the CSV file
home_sale_df = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv")

# Filter the DataFrame for homes with 3 bedrooms and 3 bathrooms
filtered_df = home_sale_df[(home_sale_df['bedrooms'] == 3) & (home_sale_df['bathrooms'] == 3)]

# Calculate the average price for each year the home was built
average_price_by_year = filtered_df.groupby('date_built')['price'].mean().round(2).reset_index()

print(average_price_by_year)



   date_built      price
0        2010  292859.62
1        2011  291117.47
2        2012  293683.19
3        2013  295962.27
4        2014  290852.27
5        2015  288770.30
6        2016  290555.07
7        2017  292676.79


In [30]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

import pandas as pd

# Create a Pandas DataFrame from the CSV file
home_sale_df = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv")

# Filter the DataFrame for homes with 3 bedrooms, 3 bathrooms, two floors, and are greater than or equal to 2,000 square feet
filtered_df = home_sale_df[(home_sale_df['bedrooms'] == 3) &
                           (home_sale_df['bathrooms'] == 3) &
                           (home_sale_df['floors'] == 2) &
                           (home_sale_df['sqft_living'] >= 2000)]

# Calculate the average price for each year the home was built
average_price_by_year = filtered_df.groupby('date_built')['price'].mean().round(2).reset_index()

print(average_price_by_year)


   date_built      price
0        2010  285010.22
1        2011  276553.81
2        2012  307539.97
3        2013  303676.79
4        2014  298264.72
5        2015  297609.97
6        2016  293965.10
7        2017  280317.58


In [31]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
import pandas as pd
import time

# Create a Pandas DataFrame from the CSV file
home_sale_df = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv")

# Filter the DataFrame for homes greater than or equal to $350,000
filtered_df = home_sale_df[home_sale_df['price'] >= 350000]

# Calculate the average "view" rating for the filtered homes
average_view_rating = filtered_df['view'].mean().round(2)

print("Average View Rating for Homes >= $350,000:", average_view_rating)

# Calculate and print the run time for this query
start_time = time.time()
# Perform the query again
average_view_rating = filtered_df['view'].mean().round(2)
end_time = time.time()

print("Average View Rating for Homes >= $350,000:", average_view_rating)
print("--- %s seconds ---" % (end_time - start_time))


Average View Rating for Homes >= $350,000: 32.26
Average View Rating for Homes >= $350,000: 32.26
--- 0.0004436969757080078 seconds ---


In [32]:
# 7. Cache the the temporary table home_sales.
import requests
import pandas as pd

# Download the CSV file using the requests library
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
response = requests.get(url)

# Save the content to a local file
with open("home_sales_revised.csv", "wb") as f:
    f.write(response.content)

# Read the CSV file into a pandas DataFrame
home_sales_df = pd.read_csv("home_sales_revised.csv")

# Create a Spark DataFrame from the pandas DataFrame
spark_home_sales_df = spark.createDataFrame(home_sales_df)

# Register the DataFrame as a temporary view
spark_home_sales_df.createOrReplaceTempView("home_sales")

# Cache the temporary table
spark.catalog.cacheTable("home_sales")




In [33]:
# 8. Check if the table is cached.
# Check if the table is cached
is_cached = spark.catalog.isCached('home_sales')

# Print the result
print(is_cached)


True


In [34]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
# 1. Cache the table 'home_sales' (if not already cached)
spark.catalog.cacheTable('home_sales')

# 2. Run the query using the cached data and measure the runtime
start_time_cached = time.time()

# Write the SQL query to filter out the view ratings with average price greater than or equal to $350,000
query_cached = """
SELECT view, ROUND(AVG(Price), 2) AS Average_Price
FROM home_sales
WHERE Price >= 350000
GROUP BY view
"""

# Execute the SQL query using the cached data and get the result as a DataFrame
result_cached = spark.sql(query_cached)

# Show the result
result_cached.show()

# Measure the runtime
print("--- Runtime with cached data: %s seconds ---" % (time.time() - start_time_cached))

# 3. Run the same query again without using the cached data and measure the runtime
start_time_uncached = time.time()

# Execute the SQL query without using the cached data and get the result as a DataFrame
result_uncached = spark.sql(query_cached)

# Show the result
result_uncached.show()

# Measure the runtime
print("--- Runtime without cached data: %s seconds ---" % (time.time() - start_time_uncached))


+----+-------------+
|view|Average_Price|
+----+-------------+
|  26|    401506.97|
|  29|    397771.65|
|  65|    736679.93|
|  19|    398953.17|
|  54|    798684.82|
|   0|    403848.51|
|  22|    402022.68|
|   7|    403005.77|
|  77|   1076205.56|
|  34|    401419.75|
|  50|     446102.7|
|  94|    1033536.2|
|  57|     734340.5|
|  43|    400929.43|
|  32|    399367.72|
|  84|   1117233.13|
|  31|    399856.95|
|  39|    399234.22|
|  98|   1053739.33|
|  25|    401298.69|
+----+-------------+
only showing top 20 rows

--- Runtime with cached data: 1.4751942157745361 seconds ---
+----+-------------+
|view|Average_Price|
+----+-------------+
|  26|    401506.97|
|  29|    397771.65|
|  65|    736679.93|
|  19|    398953.17|
|  54|    798684.82|
|   0|    403848.51|
|  22|    402022.68|
|   7|    403005.77|
|  77|   1076205.56|
|  34|    401419.75|
|  50|     446102.7|
|  94|    1033536.2|
|  57|     734340.5|
|  43|    400929.43|
|  32|    399367.72|
|  84|   1117233.13|
|  31|    

In [36]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Assuming you have the Pandas DataFrame "home_sales_df"
# Convert the Pandas DataFrame to a Spark DataFrame
# Write the Spark DataFrame to Parquet format partitioned by the "date_built" field
spark_home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("/path/to/parquet_files")
# Write the Spark DataFrame to Parquet format partitioned by the "date_built" field

spark_home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("/path/to/parquet_files")



In [37]:
# 11. Read the parquet formatted data.
# Read the Parquet formatted data from the specified location
parquet_data = spark.read.parquet("/path/to/parquet_files")

# Show the contents of the DataFrame
parquet_data.show()


+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|62c69180-ab61-47c...|2020-07-06| 395216|       4|        2|       2428|    8908|     2|         0|  20|      2017|
|273b36f4-26eb-4dd...|2021-04-27| 306223|       2|        2|       2181|   12568|     1|         0|   2|      2017|
|e2919f3b-af95-464...|2021-12-12| 152105|       2|        3|       1998|   11236|     2|         0|  21|      2017|
|1c0eeb36-0f40-479...|2022-04-07| 150817|       3|        3|       2283|    8650|     2|         0|   8|      2017|
|29a05191-0b08-4ac...|2021-03-02|1471564|       8|        3|       5088|   10228|     2|         0|  97|      2017|
|db7e9f31-e4fd-407...|2019-12-29| 152809|       3|        3|       1834|

In [40]:
# 12. Create a temporary table for the parquet data.
# Assuming you have already loaded and processed the Parquet data as Spark DataFrame
# Convert the Pandas DataFrame to a Spark DataFrame
spark_home_sales_df = spark.createDataFrame(home_sales_df)

# Create a temporary view for the Spark DataFrame
spark_home_sales_df.createOrReplaceTempView("parquet_data_temp")

# Now you can run SQL queries on the temporary view
query = "SELECT * FROM parquet_data_temp WHERE bedrooms = 3"
result = spark.sql(query)
result.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|e81aacfe-17fe-46b...|2020-06-16|      2016|181925|       3|        3|       2137|   11709|     2|         0|  22|
|2ed8d509-7372-46d...|2021-08-06|      2015|258710|       3|        3|       1918|    9666|     1|         0|  25|
|f876d86f-3c9f-42b...|2019-02-27|      2011|167864|       3|        3|       2471|   13924|     2|         0|  15|
|941bad30-eb49-4a7...|2020-05-09|      2015|229896|       3|        3|       2197|    8641|     1|         0|   3|
|dd61eb34-6589-4c0...|2021-07-25|      2016|210247|       3|        2|       167

In [41]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
# Read the parquet formatted data into a DataFrame
parquet_data_df = spark.read.parquet("/path/to/parquet_files")

# Create a temporary view for the parquet DataFrame
parquet_data_df.createOrReplaceTempView("parquet_data_temp")

# Write the SQL query to filter out view ratings with average price >= $350,000
query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM parquet_data_temp
GROUP BY view
HAVING AVG(price) >= 350000
"""

# Execute the SQL query and calculate the runtime
start_time = time.time()
result = spark.sql(query)
print("--- %s seconds ---" % (time.time() - start_time))

# Show the result
result.show()


--- 0.028130054473876953 seconds ---
+----+-------------+
|view|average_price|
+----+-------------+
|  65|    736679.93|
|  54|    798684.82|
|  77|   1076205.56|
|  94|    1033536.2|
|  57|     734340.5|
|  84|   1117233.13|
|  98|   1053739.33|
|  95|    1054325.6|
|  71|     775651.1|
|  68|    716785.44|
|  87|    1072285.2|
|  72|    780914.67|
|  58|    759764.65|
|  63|    711614.55|
|  56|     718176.4|
|  51|    788128.21|
|  52|    733780.26|
|  79|   1009565.08|
|  88|   1031719.35|
|  96|   1017815.92|
+----+-------------+
only showing top 20 rows



In [42]:
# 14. Uncache the home_sales temporary table.
# Uncache the temporary table
spark.catalog.uncacheTable("home_sales")


In [ ]:
# 15. Check if the home_sales is no longer cached
# Check if the home_sales table is cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
print("Is home_sales cached?", is_cached)



Is home_sales cached? False
